# Inspect Models

First of all, import `andes` and configure the logger. 

If logger if not configured, information will not be shown correctly.

In [1]:
import andes

andes.main.config_logger(stream_level=30)

## Inspect Model Equations

Create an empty `andes.System` object and call `prepare()` to generate the equations.

This operation may take a moment.

In [2]:
ss = andes.System()
ss.prepare()

### List all models

In [3]:
print(ss.supported_models())

Supported Groups and Models

      Group        |                          Models                          
-------------------+----------------------------------------------------------
 ACLine            | Line                                                     
 ACShort           | Jumper                                                   
 ACTopology        | Bus                                                      
 Calculation       | ACE, ACEc, COI                                           
 Collection        | Area                                                     
 DCLink            | Ground, R, L, C, RCp, RCs, RLs, RLCs, RLCp               
 DCTopology        | Node                                                     
 DG                | PVD1, ESD1                                               
 DynLoad           | ZIP, FLoad                                               
 Exciter           | EXDC2, IEEEX1, ESDC2A, EXST1, ESST3A, SEXS, IEEET1,      
                   | EX

## Check model documentation

To check the documentation for the model, print the return of `doc()` for the model instance. 

For example, the documentation for `GENCLS` can be printed with

In [4]:
print(ss.GENCLS.doc())

Model <GENCLS> in Group <SynGen>

    Classical generator model.
    
Parameters

 Name   |            Description             | Default | Unit |   Properties  
--------+------------------------------------+---------+------+---------------
 idx    | unique device idx                  |         |      |               
 u      | connection status                  | 1       | bool |               
 name   | device name                        |         |      |               
 bus    | interface bus id                   |         |      | mandatory     
 gen    | static generator index             |         |      | mandatory     
 coi    | center of inertia index            |         |      |               
 Sn     | Power rating                       | 100     |      |               
 Vn     | AC voltage rating                  | 110     |      |               
 fn     | rated frequency                    | 60      |      |               
 D      | Damping coefficient                | 0 

### Pretty print of variables

All symbols are stored in the attributes of `Model.syms`. For example,

In [5]:
ss.GENCLS.syms.xy

Matrix([
[      \delta],
[      \omega],
[         I_d],
[         I_q],
[         V_d],
[         V_q],
[      \tau_m],
[      \tau_e],
[         v_f],
[X_{ad}I_{fd}],
[      \psi_d],
[      \psi_q],
[      \theta],
[           V]])

Differential variables comes before algebraic variables.

In [6]:
ss.GENCLS.states

OrderedDict([('delta', State: GENCLS.delta, []),
             ('omega', State: GENCLS.omega, [])])

In [7]:
ss.GENCLS.algebs

OrderedDict([('Id', Algeb: GENCLS.Id, []),
             ('Iq', Algeb: GENCLS.Iq, []),
             ('vd', Algeb: GENCLS.vd, []),
             ('vq', Algeb: GENCLS.vq, []),
             ('tm', Algeb: GENCLS.tm, []),
             ('te', Algeb: GENCLS.te, []),
             ('vf', Algeb: GENCLS.vf, []),
             ('XadIfd', Algeb: GENCLS.XadIfd, []),
             ('psid', Algeb: GENCLS.psid, []),
             ('psiq', Algeb: GENCLS.psiq, [])])

### Pretty print of equations

Formatted equations are stored in each model. The following attributes of `Model.syms` are available for equation printing.

- f: differential equations
- g: algebraic equations
- df: df/dxy
- dg: dg/dxy

In [8]:
ss.GENCLS.syms.f

Matrix([
[                  2*pi*f*u*(\omega - 1)],
[u*(-D*(\omega - 1) - \tau_e + \tau_m)/M]])

In [9]:
ss.GENCLS.syms.g

Matrix([
[                 I_d*xq + \psi_d - v_f],
[                       I_q*xq + \psi_q],
[        V*u*sin(\delta - \theta) - V_d],
[        V*u*cos(\delta - \theta) - V_q],
[                   -\tau_m + \tau_{m0}],
[-\tau_e + u*(-I_d*\psi_q + I_q*\psi_d)],
[                        u*v_{f0} - v_f],
[              -X_{ad}I_{fd} + u*v_{f0}],
[           -\psi_d + u*(I_q*r_a + V_q)],
[            \psi_q + u*(I_d*r_a + V_d)],
[                -u*(I_d*V_d + I_q*V_q)],
[                -u*(I_d*V_q - I_q*V_d)]])

In [10]:
ss.GENCLS.syms.df

Matrix([
[0, 2*pi*f*u, 0, 0, 0, 0,   0,    0, 0, 0, 0, 0, 0, 0],
[0,   -D*u/M, 0, 0, 0, 0, u/M, -u/M, 0, 0, 0, 0, 0, 0]])

In [11]:
ss.GENCLS.syms.dg

Matrix([
[                        0, 0,        xq,        0,      0,      0,  0,  0, -1,  0,     1,      0,                         0,                      0],
[                        0, 0,         0,       xq,      0,      0,  0,  0,  0,  0,     0,      1,                         0,                      0],
[ V*u*cos(\delta - \theta), 0,         0,        0,     -1,      0,  0,  0,  0,  0,     0,      0, -V*u*cos(\delta - \theta), u*sin(\delta - \theta)],
[-V*u*sin(\delta - \theta), 0,         0,        0,      0,     -1,  0,  0,  0,  0,     0,      0,  V*u*sin(\delta - \theta), u*cos(\delta - \theta)],
[                        0, 0,         0,        0,      0,      0, -1,  0,  0,  0,     0,      0,                         0,                      0],
[                        0, 0, -\psi_q*u, \psi_d*u,      0,      0,  0, -1,  0,  0, I_q*u, -I_d*u,                         0,                      0],
[                        0, 0,         0,        0,      0,      0,  0,  0, -1,  0,  

### Pretty print of services

The list of services is in `services`.

In [12]:
ss.GENCLS.services

OrderedDict([('p0', ConstService: GENCLS.p0, v=[0.]),
             ('q0', ConstService: GENCLS.q0, v=[0.]),
             ('_V', ConstService: GENCLS._V, v=[0.]),
             ('_S', ConstService: GENCLS._S, v=[0.]),
             ('_I', ConstService: GENCLS._I, v=[0.]),
             ('_E', ConstService: GENCLS._E, v=[0.]),
             ('_deltac', ConstService: GENCLS._deltac, v=[0.]),
             ('delta0', ConstService: GENCLS.delta0, v=[0.]),
             ('vdq', ConstService: GENCLS.vdq, v=[0.]),
             ('Idq', ConstService: GENCLS.Idq, v=[0.]),
             ('Id0', ConstService: GENCLS.Id0, v=[0.]),
             ('Iq0', ConstService: GENCLS.Iq0, v=[0.]),
             ('vd0', ConstService: GENCLS.vd0, v=[0.]),
             ('vq0', ConstService: GENCLS.vq0, v=[0.]),
             ('tm0', ConstService: GENCLS.tm0, v=[0.]),
             ('psid0', ConstService: GENCLS.psid0, v=[0.]),
             ('psiq0', ConstService: GENCLS.psiq0, v=[0.]),
             ('vf0', ConstService: GEN

Service equations are in `Model.syms.s` For example, services of `GENCLS` is in

In [13]:
ss.GENCLS.syms.s

Matrix([
[                                                P_{0s}*\gamma_P],
[                                                Q_{0s}*\gamma_Q],
[                                                V*exp(I*\theta)],
[                                                    P_0 - I*Q_0],
[                                                    S/conj(V_c)],
[                                         I_c*(r_a + I*xq) + V_c],
[                                                  log(E/abs(E))],
[                                                 u*im(\delta_c)],
[                                V_c*u*exp(-\delta_c + 0.5*I*pi)],
[                                I_c*u*exp(-\delta_c + 0.5*I*pi)],
[                                                     re(I_{dq})],
[                                                     im(I_{dq})],
[                                                     re(V_{dq})],
[                                                     im(V_{dq})],
[u*(I_{d0}*(I_{d0}*r_a + V_{d0}) + I_{q0}*(I_{q0}*r_a